<a href="https://colab.research.google.com/github/arlingtunes/arlingtunes.github.io/blob/main/Access_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Access Metrics Made Easy: Nearest Destination and Access Count
This notebook provides a simple workflow and toolkit to calculate find access metrics. Primarily, this notebook calculates the nearest destination and the number of locations within a given threshold. The basic workflow is as follows:
1. Import data: Point data for destinations, origin geographies, and a transit cost matrix of pre-computed travel costs (eg. Minutes, Miles, etc.)
2. Spatially join destinations to origins: Based on the geospatial location, this associates each destination with an origin geography. Given that the travel cost between each origin geography is known, we can easily calculate the distance between.
3. Calculate metrics: For the nearest location, we'll simply sort the list of origins and destinations by travel time, then take the first entry. For the count within a given threshold, we can filter the list of origins and destinations by the travel time, and count the number of entries under a given threshold.

---

Getting start: Imports and a helper function for later. Here, we'll install the needed libraries (on the Colab remote machine) and import the libraries:

In [1]:
!pip install pandas geopandas access rtree pygeos access
import os
os.environ['USE_PYGEOS'] = '1'
import pandas as pd
import geopandas as gpd
import io
import urllib
from shapely.geometry import Point
# from access import Access, weights, Datasets

def dfToGdf(df, lon, lat, crs='EPSG:4326'):
  '''
    df: pandas dataframe
    lon: longitude column name
    lat: latitude column name
    crs: EPSG code or similar coordinate reference system
  '''
  return gpd.GeoDataFrame(
    df.drop([lon, lat], axis=1),
    crs=crs,
    geometry=[Point(xy) for xy in zip(df[lon], df[lat])])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.2/543.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.9 MB/s eta 0:00:00


### Public data:
We've provided a set of public datasets to help get you started. Specifically, we have travel cost matrices for US Zip codes and US Census Tracts, with the travel cost as a value in minutes. The data here is in the Apache Parquet format, an efficient format for storing tabular data.

We've also included some base geographies and population data (should you need it) and some sample destination data of Federally Qualified Health Clinics (FQHC's).

In [4]:
matrices = {
    'tract': {
        'car':'https://uchicago.box.com/shared/static/hkipez75z2p7zivtjdgsfzut4rhm6t6h.parquet',
        'bike':'https://uchicago.box.com/shared/static/cvkq3dytr6rswzrxlgzeejmieq3n5aal.parquet',
        'walk':'https://uchicago.box.com/shared/static/swggh8jxj59c7vpxzx1emt7jnd083rmh.parquet'
    },
    'zip': {
        'car':'https://uchicago.box.com/shared/static/swggh8jxj59c7vpxzx1emt7jnd083rmh.parquet',
        'bike':'https://uchicago.box.com/shared/static/7yzgf1gx3k3sacntjqber6l40m0d5aqw.parquet',
        'walk':'https://uchicago.box.com/shared/static/b3vuqijqys24z146u78dsemn0wvu8i5m.parquet',
    }
}

geographies = {
    'tract': 'https://uchicago.box.com/shared/static/kfoid6fzlbpyfecmwpe9u16cl5n89ru0.zip',
    'zip':'https://uchicago.box.com/shared/static/270ca6syxcg3dlvohhnt3ap92m4t7cxc.zip'
}
pop_data = {
    'tract':'https://uchicago.box.com/shared/static/z6xm6tre935xbc06gg4ukzgyicro26cw.csv',
    'zip': 'https://uchicago.box.com/shared/static/njjpskiuj7amcztrxjws2jfwqlv66t49.csv'
}
sample_point_data = {
    'FQHC': 'https://uchicago.box.com/shared/static/uylcq23g5z8jhvmp7cnofr074j4hwj6e.csv',
    'pharmacies': 'https://uchicago.box.com/shared/static/njjpskiuj7amcztrxjws2jfwqlv66t49.csv',
    'Maryland_ElectionPoll18': 'file:///Users/china/Downloads/HPHDLP/Marylan%202016,2018/Maryland_2018-11-06.csv',
    'opioid_treatment_facilities': 'https://raw.githubusercontent.com/GeoDaCenter/opioid-policy-scan/master/data_raw/Opioid_Treatment_Directory_Geocoded.csv',
    'moud_full': 'https://raw.githubusercontent.com/GeoDaCenter/opioid-policy-scan/master/data_final/moud/us-wide-moudsCleaned_geocoded.csv',
    'moud1': 'https://raw.githubusercontent.com/spaykin/moud-files/master/small-files/us-wide-moudsCleaned_geocoded1.csv',
    'moud2': 'https://raw.githubusercontent.com/spaykin/moud-files/master/small-files/us-wide-moudsCleaned_geocoded2.csv',
    'moud3': 'https://raw.githubusercontent.com/spaykin/moud-files/master/small-files/us-wide-moudsCleaned_geocoded3.cs'
}

geoid_cols = {
    "tract":"GEOID",
    "zip": "GEOID10"
}

Specify your preferred geographic unit and transit mode below. Or do your own thing!

In [6]:
GEOGRAPHIC_UNIT = 'zip' # 'tract' or 'zip'
TRANSIT_MODE = 'car' # 'car' or 'bike' or 'walk'

TRANSIT_MATRIX = pd.read_parquet(matrices[GEOGRAPHIC_UNIT][TRANSIT_MODE])

url = geographies[GEOGRAPHIC_UNIT]
with urllib.request.urlopen(url) as response:
    with io.BytesIO(response.read()) as zip_file:
        # Write the zip file to disk
        with open("geodata.zip", "wb") as f:
            f.write(zip_file.getbuffer())

# Read the shapefile into a GeoDataFrame
GEOGRAPHIES = gpd.read_file('geodata.zip').to_crs('EPSG:4326')
GEOGRAPHIES['FIPS'] = GEOGRAPHIES[geoid_cols[GEOGRAPHIC_UNIT]].astype('int64')

In [7]:
DESTINATIONS = dfToGdf(pd.read_csv(sample_point_data['moud_full']), 'xcoord', 'ycoord') # update your destination of interest here (i.e. point locations csv)

In [8]:
# Green comment, us here: If you're trying to set up some gravity models or other access score calculations,
# you'll probably need population data. We've got you covered. Otherwise ignore this. k? K.
POPULATION = pd.read_csv(pop_data[GEOGRAPHIC_UNIT])[['FIPS','Total Population']].iloc[1:]
POPULATION['GEOID'] = POPULATION['FIPS'].astype('int64')
GEOGRAPHIES = GEOGRAPHIES.merge(POPULATION[['FIPS','Total Population']], how="left", on="FIPS")

<ipython-input-8-fa0c5bf6f194>:3: DtypeWarning: Columns (0,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  POPULATION = pd.read_csv(pop_data[GEOGRAPHIC_UNIT])[['FIPS','Total Population']].iloc[1:]


### Spatial Join
Spatially joining points and polygons is easy. We're using the `intersects` geometric predicate here, simply meaning that if a point intersects a polygon, those two become joined or associated.

This means that we are able to see which polygon from our geographies each destination is in, and from the travel matrix, we'll know how far it is (roughly) from the other georaphies.

In [10]:
merged_destinations = gpd.sjoin(DESTINATIONS, GEOGRAPHIES[['FIPS', 'geometry']], how='inner', predicate='intersects')
merged_destinations.head()

,name1,name2,street1,street2,city,state,zip,category,countyName,source,geometry,index_right,FIPS
0,Oklahoma Treatment Services LLC,Bartlesville Rightway Medical,610 West Hensley Boulevard,NaN,Bartlesville,OK,74003,methadone,Washington,SAMHSA,POINT (-95.98404 36.75355),549,74003
1,Quapaw Counseling Services,NaN,58150 East 66 Road,NaN,Miami,OK,74354,methadone,Ottawa,SAMHSA,POINT (-94.8393 36.91948),26718,74354
2,Keetoowah Cherokee Trt Servs,NaN,2727 East Admiral Place,NaN,Tulsa,OK,74110,methadone,Tulsa,SAMHSA,POINT (-95.95045 36.16111),26906,74110
3,BHG Joplin Treatment Center,NaN,2919 East 4th Street,NaN,Joplin,MO,64801,methadone,Jasper,SAMHSA,POINT (-94.4782 37.0874),22907,64801
4,Tulsa Rightway Medical,NaN,3445 South Sheridan Road East,NaN,Tulsa,OK,74145,methadone,Tulsa,SAMHSA,POINT (-95.90032 36.11242),29323,74145


### Moins Est Plus
Less is more, let's just snag the columns we need. We'll need to join this data again to the travel matrix, so the second line gets everyone speaking the same language.

In [11]:
## Pull out the simplified columns we need for the analysis
destinations_simplified = merged_destinations[['index_right','FIPS']]
## Type correctly to merge
## Ignore that warning this is fine
destinations_simplified.loc[:,'FIPS'] = destinations_simplified['FIPS'].astype('int64')
destinations_simplified.head()

,index_right,FIPS
0,549,74003
1,26718,74354
2,26906,74110
3,22907,64801
4,29323,74145


### The Big Join
Currently, we have destinations associated with our origin geographies (if you're using the default data, census tracts and health clinics), but we need to bring it all together by joining the origins and destinations to the travel matrix. Below, we'll join the travel matrix to the destinations:

In [20]:
## Merge onto the transit matrix, giving us the distance from each origin
## to each destination
merge_transit_matrix = TRANSIT_MATRIX.merge(destinations_simplified, left_on="destination", right_on="FIPS")
merge_transit_matrix.head()

,origin,destination,minutes,index_right,FIPS
0,1001,1002,44.46,7918,1002
1,1001,1002,44.46,7918,1002
2,1001,1002,44.46,7918,1002
3,1001,1002,44.46,7918,1002
4,1001,1002,44.46,7918,1002


### Analysis Time
Let's get down to business. To begin, let's declare some variables that will help us a bit later. To start, we can define what our origin column (by default, creatively, `origin`), the destination ID column that came from the destinations data, the travel cost column, and the treshold for travel time.

In [22]:
origin_col = 'origin'
destination_id_col = 'index_right'
travel_cost_col = 'minutes'
travel_threshold = 30

### Data Cleanup
We have some weird -1000 values. Let's fix them and replace them with 999, the default null value of this travel matrix.

In [23]:
## clean up this weird bug
## then merge the data
merge_transit_matrix[travel_cost_col] = merge_transit_matrix[travel_cost_col].replace(-1000, 999)
merge_transit_matrix = merge_transit_matrix.sort_values(travel_cost_col, ascending=True)
merge_transit_matrix.minutes = merge_transit_matrix.minutes.replace(-1000, 999)
merge_transit_matrix.origin = merge_transit_matrix.origin.astype('int64')
merge_transit_matrix.head()

,origin,destination,minutes,index_right,FIPS
23259981,97420,97420,0.0,2255,97420
14987602,34134,34134,0.0,21187,34134
14987601,34134,34134,0.0,21187,34134
14987600,34134,34134,0.0,21187,34134
14987599,34134,34134,0.0,21187,34134


### Nearest location
To get the nearest location, sort the values by lowest cost then filter for  the first appearance of each origin ID. This means we'll get the first time that origin shows up, sorted by the lowest travel cost.

We'll use pandas `.duplicated()` function with the not (`~`) operator before it.

In [24]:
time_to_nearest = merge_transit_matrix[~merge_transit_matrix.origin.duplicated()][[origin_col, travel_cost_col]]
time_to_nearest.head()

,origin,minutes
23259981,97420,0.0
14987602,34134,0.0
7056344,10930,0.0
23609658,98409,0.0
14987212,34119,0.0


### Count in Threshold
For getting the count of destinations within a given threshold (by default, 30 minutes), we can chain a couple functions here from pandas.

First, we'll filter the `travel_costs` dataframe for costs that are less than or equal to the threshold. Then group by the origin column, giving us sets of rows that share the same origin ID, and then count those columns, giving us the number of rows for each origin ID with a travel cost under our treshold.

We'll re-label some columns for easy reference.

In [25]:
## For count, we simple filter for the cost under a given threshold
## Then groupby by and count the results
count_within_threshold = merge_transit_matrix[merge_transit_matrix[travel_cost_col] <= travel_threshold] \
  .groupby(origin_col).count() \
  .reset_index()[[origin_col, travel_cost_col]]
count_within_threshold.columns = [origin_col, f"count within {travel_threshold}"]

### Merge Results
Now, we can merge our two findings into an easy, breezy, beautiful dataframe.

In [26]:
merged_metrics = count_within_threshold.merge(time_to_nearest, on=origin_col, how="outer")
merged_metrics.head()

,origin,count within 30,minutes
0,1001,266.0,8.04
1,1002,113.0,0.00
2,1003,104.0,0.00
3,1005,29.0,0.00
4,1007,97.0,0.00


### Cleanup
One last edge case to handle here: It is possible some origins are not within 30 minutes of a destination, meaning some of the data will be null. Or, we might have lost some origins from the full geographies dataset.

While not the end of the world, we can clean this up here before shipping of results to our (soon to be disgruntled) data scientist colleagues.

The below finds the missing origin IDs and fills them in, giving us the revered `findings` dataframe.

🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉

In [29]:
## To clean up any missing data, we can check back with our origin list
analyzed_origins = list(merged_metrics[origin_col])
missing_origins = [o for o in GEOGRAPHIES.FIPS if o not in analyzed_origins]

## Then, fill the missing data
missing_data = []
for o in missing_origins:
    missing_entry = {}
    missing_entry[origin_col] = o
    missing_entry[f"count within {travel_threshold}"]=0
    missing_entry[travel_cost_col]=None
    missing_data.append(missing_entry)
missing_df = pd.DataFrame(missing_data)

## and concatenate results
findings = pd.concat([merged_metrics, missing_df])
# Fill any null values with 0 for count within
findings['count within 30'] = findings['count within 30'].fillna(0).astype(int)
# Replace error value "999" in matrices with blanks
findings['minutes'] = findings['minutes'].replace(999.0, None)
findings.head()

<ipython-input-29-ac3376cf5973>:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  findings = pd.concat([merged_metrics, missing_df])


,origin,count within 30,minutes
0,1001,266,8.04
1,1002,113,0.00
2,1003,104,0.00
3,1005,29,0.00
4,1007,97,0.00


### What's Next?
Well, now you could take this data and export it as a CSV, or join it back to the geographies and visualize it, or try running this analysis with some different data. Ball is in your court, you got this!


In [30]:
# Export to csv
findings.to_csv('moud1_access.csv', index = False)

#### Who me?
Oh, sure. If you want to see more, check out [the Covid Atlas](uscovidatlas.org) or [HEROP](https://herop.ssd.uchicago.edu/), [CSDS](https://spatial.uchicago.edu/), or [my github](https://github.com/nofurtherinformation).

Got Q's? Liked this little notebook? Feel free to drop me a line a dhalpern at uchicago dot edu, or on Twitter at [dchalpern](https://twitter.com/dchalpern).